In [1]:
library(readxl)
library(dplyr)
library(DTComPair)
library(summarytools)
library(vcd)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: gee

Loading required package: PropCIs

Loading required package: grid



In [2]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 19042)

Matrix products: default

locale:
[1] LC_COLLATE=English_United Kingdom.1252 
[2] LC_CTYPE=English_United Kingdom.1252   
[3] LC_MONETARY=English_United Kingdom.1252
[4] LC_NUMERIC=C                           
[5] LC_TIME=English_United Kingdom.1252    

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] vcd_1.4-8          summarytools_1.0.0 DTComPair_1.0.3    PropCIs_0.3-0     
[5] gee_4.13-20        dplyr_1.0.7        readxl_1.3.1      

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-6       zoo_1.8-9          tidyselect_1.1.1   repr_1.1.3        
 [5] purrr_0.3.4        pander_0.6.4       lattice_0.20-45    tcltk_4.1.2       
 [9] colorspace_2.0-2   vctrs_0.3.8        generics_0.1.0     htmltools_0.5.2   
[13] base64enc_0.1-3    utf8_1.2.2         rlang_0.4.12       p

# 1. Cohen weighted kappa & inter-rater agreement plots

[Weighted Kappa](https://www.datanovia.com/en/lessons/weighted-kappa-in-r-for-two-ordinal-variables/)   
[Inter-rater agreement plot](https://www.datanovia.com/en/lessons/inter-rater-agreement-chart-in-r/)

# 1.1. Non-blinded test vs. Blinded test

In [2]:
# pT3a as locally advanced

my_data <- read_excel(".data/BIMJF3aLA.xlsx") 
# # for analysis with pT3a as not locally advanced 
# my_data <- read_excel(".data/BIMJF3aNOLA.xlsx") 

In [3]:
colnames(my_data)

[1] "TEST1NB" "TEST2BL" "TEST3HP" "LATEST1" "LATEST2" "LATEST3"

In [4]:
# Not blinded vs. Blinded
notbl_bl <- with(my_data, ctable(TEST1NB, TEST2BL))$cross_table[1:4,1:4]

In [5]:
dimnames(notbl_bl) <- list(
  Nonblinded = c("T0-3a", "T3b", "T4a", "T4b"),
  Blinded = c("T0-3a", "T3b", "T4a", "T4b")
)
notbl_bl

          Blinded
Nonblinded T0-3a T3b T4a T4b
     T0-3a    99   5   4   0
     T3b       8   4   2   0
     T4a       2   0   4   2
     T4b       0   0   0   4

In [6]:
# Computing kappa
res.k <- Kappa(notbl_bl, weights='Equal-Spacing')
summary(res.k)

            value     ASE     z  Pr(>|z|)
Unweighted 0.4824 0.08596 5.611 2.007e-08
Weighted   0.6188 0.08193 7.553 4.252e-14

Weights:
          [,1]      [,2]      [,3]      [,4]
[1,] 1.0000000 0.6666667 0.3333333 0.0000000
[2,] 0.6666667 1.0000000 0.6666667 0.3333333
[3,] 0.3333333 0.6666667 1.0000000 0.6666667
[4,] 0.0000000 0.3333333 0.6666667 1.0000000

### Confidence intervals

In [7]:
confint(res.k)

,lwr,upr
Unweighted,0.3138832,0.6508464
Weighted,0.4582224,0.7793683


In [8]:
options(jupyter.plot_mimetypes = c('image/png', 'image/jpeg', 'image/svg'))
png(filename="notbl_bl.png", 
    width=640, 
    height=640, 
    pointsize=12)
p <- agreementplot(notbl_bl)
dev.off()

png 
  2

### B-coefficient (Bandgiwala)

In [9]:
# Show the Bangdiwala agreement strength statistics
unlist(p)[1:2]

Bangdiwala Bangdiwala_Weighted 
          0.8206132           0.9291137

## <u> Report example</u>
Weighted kappa (kw) with linear weights (Cicchetti and Allison 1971) was computed to assess if there was agreement between two clinical doctors in staging prostate cancer blinded vs unblinded test. 134 participants were enrolled and were classified by each of the two doctors into 4 stages of cancer.

There was a statistically significant agreement between the two doctors, kw = 0.61 (95% CI, 0.46 to 0.78), p < 0.0001. The strength of agreement was classified as moderate according to Fleiss et al. (2003).

# 1.2. Non-blinded vs. pathology

In [10]:
# Not blinded vs. patholohical examination
ptm <- my_data
ptm$TEST3HP[ptm$TEST3HP == 6] <- 2
notbl_ptm <- with(ptm, ctable(TEST1NB, TEST3HP))$cross_table[1:4,1:4]
dimnames(notbl_ptm) <- list(
  Nonblinded = c("T0-3a", "T3b", "T4a", "T4b"),
  Pathology = c("T0-3a", "T3b", "T4a", "T4b")
)
notbl_ptm

          Pathology
Nonblinded T0-3a T3b T4a T4b
     T0-3a    93   1  14   0
     T3b       6   5   3   0
     T4a       1   0   5   2
     T4b       0   0   1   3

In [11]:
dimnames(notbl_ptm)

$Nonblinded
[1] "T0-3a" "T3b"   "T4a"   "T4b"  

$Pathology
[1] "T0-3a" "T3b"   "T4a"   "T4b"

In [12]:
# Computing kappa
res.k2 <- Kappa(notbl_ptm, weights='Equal-Spacing')
summary(res.k2)

            value     ASE     z  Pr(>|z|)
Unweighted 0.4537 0.07731 5.869 4.396e-09
Weighted   0.5248 0.08022 6.543 6.042e-11

Weights:
          [,1]      [,2]      [,3]      [,4]
[1,] 1.0000000 0.6666667 0.3333333 0.0000000
[2,] 0.6666667 1.0000000 0.6666667 0.3333333
[3,] 0.3333333 0.6666667 1.0000000 0.6666667
[4,] 0.0000000 0.3333333 0.6666667 1.0000000

### Confidence intervals

In [13]:
confint(res.k2)

,lwr,upr
Unweighted,0.3021732,0.6052234
Weighted,0.3676040,0.6820414


In [14]:
options(jupyter.plot_mimetypes = c('image/png', 'image/jpeg', 'image/svg'))
png(filename="figures/notbl_ptm.png", 
    width=640, 
    height=640, 
    pointsize=12)
p <- agreementplot(notbl_ptm)
dev.off()

png 
  2

### B-coefficient (Bandgiwala)

In [15]:
# Show the Bangdiwala agreement strength statistics
unlist(p)[1:2]

Bangdiwala Bangdiwala_Weighted 
          0.7853535           0.8466811

# 1.3. Blinded vs. pathology

In [16]:
# Not blinded vs. patholohical examination
bl_ptm <- with(ptm, ctable(TEST2BL, TEST3HP))$cross_table[1:4,1:4]
dimnames(bl_ptm) <- list(
  Blinded = c("T0-3a", "T3b", "T4a", "T4b"),
  Pathology = c("T0-3a", "T3b", "T4a", "T4b")
)
bl_ptm

       Pathology
Blinded T0-3a T3b T4a T4b
  T0-3a    93   2  14   0
  T3b       4   3   2   0
  T4a       2   1   5   2
  T4b       1   0   2   3

In [17]:
# Computing kappa
res.k3 <- Kappa(bl_ptm, weights='Equal-Spacing')
summary(res.k3)

            value     ASE     z  Pr(>|z|)
Unweighted 0.4037 0.07744 5.213 1.854e-07
Weighted   0.4903 0.08177 5.997 2.015e-09

Weights:
          [,1]      [,2]      [,3]      [,4]
[1,] 1.0000000 0.6666667 0.3333333 0.0000000
[2,] 0.6666667 1.0000000 0.6666667 0.3333333
[3,] 0.3333333 0.6666667 1.0000000 0.6666667
[4,] 0.0000000 0.3333333 0.6666667 1.0000000

### Confidence intervals

In [19]:
# Confidence intervals
confint(res.k3)

,lwr,upr
Unweighted,0.2519537,0.5555218
Weighted,0.3300692,0.6505964


In [20]:
options(jupyter.plot_mimetypes = c('image/png', 'image/jpeg', 'image/svg'))
png(filename="bl_ptm.png", 
    width=640, 
    height=640, 
    pointsize=12)
p <- agreementplot(bl_ptm)
dev.off()

png 
  2

### B-coefficient (Bandgiwala)

In [18]:
# Show the Bangdiwala agreement strength statistics
unlist(p)[1:2]

Bangdiwala Bangdiwala_Weighted 
          0.7853535           0.8466811

# 2. Binary tests comparison

[DTComPair](https://cran.r-project.org/web/packages/DTComPair/DTComPair.pdf)  
Significance level **alpha = 0.05**.
- **sensitivity** - sensitivity 
- **specificity** - specificity 
- **ppv** - positive predictive value
- **npv** -  negative predictive value (est)
- **pdlr** - positive diagnostic likelihood ratio
- **ndlr** - negative diagnostic likelihood ratio  
All values have estimate (Est.), standard error (SE), lower and upper confidence intervals (Lower CL & Upper CL). 

## 2.1. Comparing two tests with 3a as locally advanced

In [19]:
la_ctable <- my_data[, 4:6] - 1

In [23]:
tab <- tab.paired(d=LATEST3, y1=LATEST1, y2=LATEST2, data=la_ctable, testnames=c("Nonblinded", "Blinded"))

In [24]:
acc.paired(tab=tab, alpha=0.05)

Diagnostic accuracy of test 'Nonblinded'

(Estimates, standard errors and 95%-confidence intervals)

                 Est.         SE  Lower CL  Upper CL
Sensitivity 0.4074074 0.06686451 0.2763554 0.5384594
Specificity 0.9500000 0.02436699 0.9022416 0.9977584
PPV         0.8461538 0.07075894 0.7074689 0.9848388
NPV         0.7037037 0.04393859 0.6175856 0.7898218

           Est.  SE (log)  Lower CL   Upper CL
PDLR  8.1481481 0.5142334 2.9740114 22.3241644
NDLR  0.6237817 0.1157125 0.4972079  0.7825773

----------------------------------------------------------
Diagnostic accuracy of test 'Blinded'

(Estimates, standard errors and 95%-confidence intervals)

                 Est.         SE  Lower CL  Upper CL
Sensitivity 0.3888889 0.06634008 0.2588647 0.5189131
Specificity 0.9500000 0.02436699 0.9022416 0.9977584
PPV         0.8400000 0.07332121 0.6962931 0.9837069
NPV         0.6972477 0.04400723 0.6109951 0.7835003

           Est.  SE (log)  Lower CL   Upper CL
PDLR  7.7777778 0.516

## 2.2. Comparing two tests with 3a as not locally advanced

In [25]:
nola_ctable <-read_excel("./data/BIMJF3aNOLA.xlsx")[, 4:6] - 1 #pT3a as not locally advanced

In [26]:
tab <- tab.paired(d=LATEST3, y1=LATEST1, y2=LATEST2, data=nola_ctable, testnames=c("Nonblinded", "Blinded"))

In [27]:
acc.paired(tab=tab, alpha=0.05)

Diagnostic accuracy of test 'Nonblinded'

(Estimates, standard errors and 95%-confidence intervals)

                 Est.         SE  Lower CL  Upper CL
Sensitivity 0.5588235 0.08515380 0.3919251 0.7257219
Specificity 0.9300000 0.02551470 0.8799921 0.9800079
PPV         0.7307692 0.08698929 0.5602734 0.9012651
NPV         0.8611111 0.03327756 0.7958883 0.9263339

           Est.  SE (log)  Lower CL   Upper CL
PDLR  7.9831933 0.3950658 3.6804088 17.3163849
NDLR  0.4743833 0.1949554 0.3237301  0.6951455

----------------------------------------------------------
Diagnostic accuracy of test 'Blinded'

(Estimates, standard errors and 95%-confidence intervals)

                 Est.         SE  Lower CL  Upper CL
Sensitivity 0.5294118 0.08560081 0.3616373 0.6971863
Specificity 0.9300000 0.02551470 0.8799921 0.9800079
PPV         0.7200000 0.08979978 0.5439957 0.8960043
NPV         0.8532110 0.03389704 0.7867740 0.9196480

           Est.  SE (log)  Lower CL   Upper CL
PDLR  7.5630252 0.398

## 2.3. Parameters for imaging study vs. histopathologic examination

### 2.3.1. pT3a as locally advanced

In [52]:
la_imgtable <-read_excel("./data/IMGHP3aLA.xlsx") - 1 

In [58]:
tab <- tab.1test(d=TEST2HP, y=TEST1IMG, data=la_imgtable, testname="Imaging")

In [59]:
acc.1test(tab=tab)

Diagnostic accuracy of test 'Imaging'

(Estimates, standard errors and 95%-confidence intervals)

                 Est.         SE  Lower CL  Upper CL
Sensitivity 0.6190476 0.07493293 0.4721818 0.7659135
Specificity 0.7068966 0.05976878 0.5897519 0.8240412
PPV         0.6046512 0.07456044 0.4585154 0.7507869
NPV         0.7192982 0.05951681 0.6026474 0.8359490

           Est.  SE (log)  Lower CL  Upper CL
PDLR  2.1120448 0.2371374 1.3269403 3.3616685
NDLR  0.5389082 0.2141012 0.3542187 0.8198948

In [60]:
tab

Binary diagnostic test 'Imaging'

          Diseased Non-diseased Total
Test pos.       26           17    43
Test neg.       16           41    57
Total           42           58   100

### 2.3.1. pT3a as not locally advanced

In [65]:
nola_imgtable <-read_excel("./data/IMGHP3aNOLA.xlsx") - 1

In [66]:
tab <- tab.1test(d=TEST2HP, y=TEST1IMG, data=nola_imgtable, testname="Imaging")

In [67]:
acc.1test(tab=tab)

Diagnostic accuracy of test 'Imaging'

(Estimates, standard errors and 95%-confidence intervals)

                 Est.         SE  Lower CL  Upper CL
Sensitivity 0.6296296 0.09293490 0.4474806 0.8117787
Specificity 0.6438356 0.05604686 0.5339858 0.7536855
PPV         0.3953488 0.07456044 0.2492131 0.5414846
NPV         0.8245614 0.05037750 0.7258233 0.9232995

           Est.  SE (log) Lower CL  Upper CL
PDLR  1.7678063 0.2157531 1.158205 2.6982615
NDLR  0.5752561 0.2655954 0.341811 0.9681362

In [68]:
tab

Binary diagnostic test 'Imaging'

          Diseased Non-diseased Total
Test pos.       17           26    43
Test neg.       10           47    57
Total           27           73   100

# Citations

1. R Core Team (2020). R: A language and environment for statistical computing. R Foundation for Statistical Computing, Vienna, Austria. URL https://www.R-project.org/.
2. Hadley Wickham and Jennifer Bryan (2019). readxl: Read Excel Files. R package version 1.3.1. https://CRAN.R-project.org/package=readxl
3. Hadley Wickham, Romain François, Lionel Henry and Kirill Müller (2021). dplyr: A Grammar of Data Manipulation. R package version 1.0.5. https://CRAN.R-project.org/package=dplyr
4. Christian Stock, Thomas Hielscher (2014). DTComPair: comparison of binary diagnostic tests in a paired study design. R package version 1.0.3. URL: http://CRAN.R-project.org/package=DTComPair
5. Dominic Comtois (2021). summarytools: Tools to Quickly and Neatly Summarize Data. R package version 0.9.9.
6.  David Meyer, Achim Zeileis, and Kurt Hornik (2020). vcd: Visualizing Categorical Data. R package version 1.4-8.
